In [2]:
import pandas as pd 
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text

- Participation _ campaign_id

In [11]:
create_table_follower = text('''CREATE TABLE IF NOT EXISTS followers (
    follower_id INTEGER PRIMARY KEY AUTOINCREMENT, 
    participation_id INTEGER,
    cookies TEXT,
    join_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (participation_id) REFERENCES campaign_participation(participation_id) ON DELETE CASCADE
);''')

In [12]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

In [13]:
# Assuming 'engine' is already defined and imported from SQLAlchemy setup
with engine.connect() as connection:
        connection.execute(create_table_follower)
        print("table created successfully.")

2024-04-18 11:11:14,258 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:11:14,259 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS followers (
    follower_id INTEGER PRIMARY KEY AUTOINCREMENT, 
    participation_id INTEGER,
    cookies TEXT,
    join_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (participation_id) REFERENCES campaign_participation(participation_id) ON DELETE CASCADE
);
2024-04-18 11:11:14,259 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()
table created successfully.
2024-04-18 11:11:14,265 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
pd.read_sql('SELECT * FROM followers',engine)

2024-04-18 11:11:29,476 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:11:29,476 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM followers")
2024-04-18 11:11:29,477 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:11:29,478 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM followers")
2024-04-18 11:11:29,478 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:11:29,479 INFO sqlalchemy.engine.Engine SELECT * FROM followers
2024-04-18 11:11:29,479 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:11:29,483 INFO sqlalchemy.engine.Engine ROLLBACK


,follower_id,participation_id,cookies,join_date


In [17]:
def add_follower(participation_id, **kwargs):
     data = {'participation_id': participation_id, **kwargs}
     columns = ', '.join(data.keys())
     placeholders = ', '.join(f":{key}" for key in data.keys())

     insert_query = text(f"""
            INSERT INTO followers ({columns})
            VALUES ({placeholders});
        """)
     with engine.begin() as connection:
        connection.execute(insert_query, data)
        print(f"Follower added")


def modify_follower(follower_id, **kwargs):
    updates = ', '.join(f"{key} = :{key}" for key in kwargs)
    update_query = text(f"""
        UPDATE campaign_participation
        SET {updates}
        WHERE follower = :follower_id;
    """)
    kwargs['follower_id'] = follower_id  # Include for WHERE clause
    with engine.begin() as connection:
        result = connection.execute(update_query, kwargs)
        if result.rowcount > 0:
            print("Follower updated")
        else:
            print(f"No follower found with ID {follower_id}.")



def get_follower(**kwargs):
    if not kwargs:
        print("No parameters provided to search for participation.")
        return None

    # Constructing the WHERE clause based on kwargs
    conditions = [f"{key} = :{key}" for key in kwargs]
    where_clause = " AND ".join(conditions)
    select_query = text(f"SELECT * FROM campaign_participation WHERE {where_clause}")

    print(select_query)  # Optional: Print the query for debugging
    with engine.connect() as connection:
        result = connection.execute(select_query, kwargs).fetchone()
        print(result)  # Optional: Print the raw result for debugging
        
        if result:
            # Assuming we know the columns in order of the table
            follower_dict = {
                'follower_id': result[0],
                'participation_id': result[1],
                'cookies': result[2],
                'join_date': result[3],
            }
            
            return follower_dict
        else:
            print("follower not found.")
            return None

In [18]:
add_follower(1)

2024-04-18 11:38:42,161 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:38:42,162 INFO sqlalchemy.engine.Engine 
            INSERT INTO followers (participation_id)
            VALUES (?);
        
2024-04-18 11:38:42,162 INFO sqlalchemy.engine.Engine [generated in 0.00067s] (1,)
Follower added
2024-04-18 11:38:42,171 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
pd.read_sql("SELECT * FROM followers",engine)

2024-04-18 11:39:01,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:39:01,232 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM followers")
2024-04-18 11:39:01,233 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:39:01,234 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM followers")
2024-04-18 11:39:01,235 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:39:01,235 INFO sqlalchemy.engine.Engine SELECT * FROM followers
2024-04-18 11:39:01,236 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:39:01,239 INFO sqlalchemy.engine.Engine ROLLBACK


,follower_id,participation_id,cookies,join_date
0,1,1,None,2024-04-18 09:38:42
